# Code

In [1]:
import os
import re
import sys
import traceback
from collections import deque
from typing import List, Tuple, TextIO, Dict
from collections import UserString

import nbformat
from nbformat.sign import NotebookNotary
from nbformat.notebooknode import NotebookNode, from_dict

import testmynb

notebook_notary = NotebookNotary() 

class CodeCell(UserString):
    """
    A class for Jupyter Notebook code cell.
    
    Attributes
    ----------
    ignore : bool    
        Whether the cell magic line contained the `-t` option.
    name : str    
        The user defined name of the test cell block.
    """    

    def __init__(self, data, notebook = None):
        super().__init__(data['source'])
        magic_args = self.parse_cell_magic(self)
        if '-n' in magic_args:
            self.ignore = True
            magic_args.remove('-n')
        else:
            self.ignore = False
            
        try:
            self.name = magic_args[0]
        except IndexError:
            self.name = 'unnamed'
        
        self.data = re.sub(r'\%\%testcell.*\n', '', self.data, count = 1)
        
        self.notebook = notebook
    
    @staticmethod
    def parse_cell_magic(source: str) -> List[str]:
        line = source.split('\n')[0]
        args = re.split(r' +', line)
        del args[0] # delete '%%testcell'
        return args
    
    def __str__(self):
        return self.data
    
    def __repr__(self):
        return self.name
        

class Notebook(NotebookNode):
    """
    A class used to read the Jupyter Notebook
    
    Parameters
    ----------
    ipynb : TextIO
        Path to the `.ipynb` file.
    
    Attributes
    ----------
    ipynb : TestIO
        Absolute path to the `.ipynb` file that was given to instantiate the instance.
    
    name : str
        Name of the `.ipynb` file.
    
    trusted : bool
        Whether the Notebook is `Trusted` or not for the user.
    
    nbformat : str
        The Jupyter Notebook format number.
    
    Methods
    -------
    extract_codes()
        Returns a list of code cells with the `%%testcell` cell magic. 
    
    """
    def __init__(self, ipynb: TextIO):
        _notebook: NotebookNode = nbformat.read(ipynb, as_version=4)
        value: dict = from_dict(_notebook)

        # self is a dict. If you look at the MRO, NotebookNode is a dict. 
        # Think of the below as two dicts (`self` and `value`) merging
        # (where `self` happens to be an empty dict).
        dict.__init__(self, value)
        
        
        
        self.__dict__['ipynb'] = os.path.abspath(ipynb)
        self.__dict__['name'] = os.path.basename(ipynb)
        self.__dict__['env'] = dict()
        
        self.__dict__['tests'] = self.extract_codes()
        
        self.__dict__['result'] = None
        self.__dict__['stack'] = None
    
    @property
    def trusted(self):
        return notebook_notary.check_signature(self)
    
    def extract_codes(self) -> List[CodeCell]:
        code_list = list()
        for cell in self.cells:
            if cell.cell_type=='code' \
                and re.match(r'^%%testcell', cell.source):
                code_list.append(CodeCell(cell, self))
        return code_list
    
    
    def __hash__(self):
        return hash(self.name)
    
    def __eq__(self, other):
        if isinstance(other, str):
            return self.name == other
        return self.name == other.name
    
    def __repr__(self):
        return self.name

    def __call__(self):
        result = list()
        stack = dict()
        for cell in self.tests:

            status, err = self.run_test(cell, self.env)
            result.append(status)
            if status!='.':
                stack[cell] = {'status': status, 'traceback': err}
        
        self.__dict__['result'] = ''.join(result)
        self.__dict__['stack'] = stack
    
    def get_error_stack(self):
        error_stack = dict()
        for cell, err in notebook.stack.items():
            if err['status']=='E':
                error_stack[cell] = err['traceback']
        return error_stack
    
    def get_fail_stack(self):
        fail_stack = dict()
        for cell, err in notebook.stack.items():
            if err['status']=='F':
                fail_stack[cell] = err['traceback']
        return fail_stack
    
    @staticmethod
    def run_test(codecell: CodeCell, env: dict) -> Tuple[str, str]:
        """

        Returns
        -------
        str : '.', 'F', 'E'
            '.' == Passed
            'F' == Failed
            'E' == Error
        str
            Traceback message. Empty string if the test passed. 
        """
        try:
            exec(str(codecell), env)
            status = '.'
        except AssertionError:
            status = 'F'
            err = traceback.format_exc()
            # This traceback function only works within the exception block.
        except:
            status = 'E'
            err = traceback.format_exc()
        else:
            return status, ''
        return status, err
        
    
class TestHandler:
    def __init__(self, *notebooks):
        self.notebooks = notebooks
            
    def __call__(self):
        
        for nb in self.notebooks:
            nb()
        
        notebook_count = len(self.notebooks)
        test_count = sum([len(nb.extract_codes()) for nb in self.notebooks])
        py_ver = re.sub(r'\s.*', '', sys.version)
        
        head_message = f' Test My Notebook ({testmynb.__version__}) '
        col, _ = os.get_terminal_size()
        num_equals = (col - len(head_message)) // 2
        equals_sign = num_equals * '='

        print(
            f'{equals_sign}{head_message}{equals_sign}\n'
            f'Platform {sys.platform}\n'
            f'Python {py_ver}\n'
            f'Working Directory: {os.getcwd()}\n'
            '\n'
            f'{test_count} test cells across {notebook_count} notebook(s) detected.\n'
            '\n'
            'Notebooks:'
            ''
        )
        for nb in self.notebooks:
            trust = 'Trusted' if nb.trusted else 'Untrusted'
            print(f'{trust} {nb.name}: {nb.result}')
        print('\n')
        errors = self.collect_errors()
        fails = self.collect_fails()
        
        if errors:
            head_message = ' Errored Test(s) '
            num_equals = (col - len(head_message)) // 2
            equals_sign = num_equals * '='
            print(f'{equals_sign}{head_message}{equals_sign}\n')
            for cell, err in errors.items():
                print(f'---- {cell.notebook}: {cell.name} ----\n')
                print(cell)
                print('-----------------------------------------')
                print(err)
                #print(f'---- {cell.notebook}: {cell.name} ----\n')
                print('\n\n\n\n\n')
                
        if fails:
            head_message = ' Failed Test(s) '
            num_equals = (col - len(head_message)) // 2
            equals_sign = num_equals * '='
            print(f'{equals_sign}{head_message}{equals_sign}\n')
            for cell, err in fails.items():
                print(f'---- {cell.notebook}: {cell.name} ----\n')
                print(cell)
                print('-----------------------------------------')
                print(err)
                #print(f'---- {cell.notebook}: {cell.name} ----\n') 
                print('\n\n\n\n\n')

    def collect_errors(self):
        errors = dict()
        for nb in self.notebooks:
            errors.update(nb.get_error_stack())
            
        return errors

    def collect_fails(self):
        fails = dict()
        for nb in self.notebooks:
            fails.update(nb.get_fail_stack())
            
        return fails
    
        
notebook = Notebook('test.ipynb')
notebook2 = Notebook('test_notebook2.ipynb')

handler = TestHandler(notebook, notebook2)
handler.notebooks
handler()

============================================================================= Test My Notebook (0.0.1) =============================================================================
Platform darwin
Python 3.6.4
Working Directory: /Users/YoungChanPark/nbtest/test

8 test cells across 2 notebook(s) detected.

Notebooks:
Trusted test.ipynb: .F.E
Trusted test_notebook2.ipynb: ....


================================================================================== Errored Test(s) ==================================================================================

---- test.ipynb: real_failing_test ----

from testmynb.magic import intentional_error_func

assert False == intentional_error_func()
-----------------------------------------
Traceback (most recent call last):
  File "<ipython-input-1-f868468a290d>", line 174, in run_test
    exec(str(codecell), env)
  File "<string>", line 3, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/testmynb/

# CLI Tool Output

# Playground

In [2]:
from testmynb import TestHandler, Notebook

In [3]:
notebook = Notebook('test.ipynb')
notebook2 = Notebook('test_notebook2.ipynb')

handler = TestHandler(notebook, notebook2)
handler.notebooks
#handler()

(test.ipynb, test_notebook2.ipynb)

In [13]:
@click.argument()

<function genericpath.isfile>

In [55]:
import click

click.Path()

click.types.Path

In [48]:
import os

def find_notebooks(*args):
    notebooks = list()
    if len(args):
        for path in args:
            if os.path.isfile(path):
                notebooks.append(path)
            elif os.path.isdir(path):
                notebooks.extend(_recursive_find_notebooks(path))
    else: 
        notebooks = _recursive_find_notebooks(os.getcwd())
    return notebooks

def _recursive_find_notebooks(path):
    notebooks = list()
    for root, dirs, files in os.walk(path):
        for file in files:
            if '.ipynb_checkpoints' in root:
                continue
            if re.match(r'^test_.+\.ipynb', file):
                notebooks.append(os.path.join(root, file))
                
    return notebooks

In [53]:
test_notebooks = find_notebooks()
handler = TestHandler(*[Notebook(nb) for nb in test_notebooks])
handler.notebooks
handler()

=========================================================================================================================== Test My Notebook (0.0.1) ===========================================================================================================================
Platform darwin
Python 3.6.4
Working Directory: /Users/YoungChanPark/nbtest/test

8 test cells across 2 notebook(s) detected.

Notebooks:
Trusted test_notebook1.ipynb: .F.E
Trusted test_notebook2.ipynb: ....


=============================================================================================================================== Errored Test(s) ===============================================================================================================================

---- test_notebook1.ipynb: real_failing_test ----

from testmynb.magic import intentional_error_func

assert False == intentional_error_func()
-----------------------------------------
Traceback (most recent call last):
  File "/Library/Framewo